In [1]:
import pandas as pd
import requests
import time
from datetime import datetime, timedelta
import re
import sqlalchemy
import MetaTrader5 as mt

In [2]:
def get_realtime(ticker='EURUSD.i', tf='5m', initial_bar = 3000):
        count = 0
        
        while True:
                # global bars, df
                # bars = exchange.fetch_ohlcv(coin , timeframe = tf, limit = initial_bar)
                try:
                        #Fetch OHLCV
                        df = pd.DataFrame(mt.copy_rates_from_pos(ticker,  mt.TIMEFRAME_M5, 0, 99999  ) )

                        df.columns = ['datetime', 'open', 'high', 'low', 'close', 'tickvol', 'spread','real_volume']
                        df = df[['datetime', 'open', 'high', 'low', 'close', 'spread','tickvol']]
                        df['tickvol'] = df['tickvol'].astype('int')
                        df['datetime'] = pd.to_datetime(df['datetime'], unit='s') #+ pd.Timedelta( hours = 4)
                        
                        now = pd.Timestamp.now().floor('min')

                        tframe =  '5m'
                        tf_t = int( re.findall('[0-9]+', tframe)[0] )
                        now = now.replace(minute= int(now.minute / tf_t)*tf_t ,second= 0,microsecond =0 )
                        
                        try :
                                print(now)
                                # df.set_index('datetime').loc[ now ]
                                # df.set_index('datetime').loc[now]
                                print('Found the bar : ' + str(now))
                                break
                        
                        except KeyError:
                                
                                print('Not found the bar.')
                                count+=1
                                if count == 36:
                                        break
                                time.sleep(1-time.time()%1)
                except:
                        print('Fetching fail.')
                        # pass
                        time.sleep(1)

        return [df , ticker ,pd.to_datetime(now) ]

In [3]:
mt.initialize()

True

In [4]:
from sqlalchemy import create_engine

In [5]:
# Create an SQLite database engine
engine = create_engine('sqlite:///MT5_E8.db', echo=True)

# Convert dataframe to SQL table
def dataframe_to_sql(dataframe, table_name, engine):
    dataframe.to_sql(table_name, con=engine, if_exists='append', index=False)


In [6]:
# get all symbols
symbols=mt.symbols_get()
print('Symbols: ', len(symbols))
count=0
# display the first five ones
for s in symbols:
    df = get_realtime(ticker = str(s.name) )[0]
    dataframe_to_sql(df,  str(s.name) , engine)
    print( str(s.name) )
 

Symbols:  97
2023-06-27 19:25:00
Found the bar : 2023-06-27 19:25:00
2023-06-27 19:25:07,732 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-27 19:25:07,737 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("ASX200")
2023-06-27 19:25:07,738 INFO sqlalchemy.engine.Engine [raw sql] ()


2023-06-27 19:25:09,155 INFO sqlalchemy.engine.Engine INSERT INTO "ASX200" (datetime, open, high, low, close, spread, tickvol) VALUES (?, ?, ?, ?, ?, ?, ?)
2023-06-27 19:25:09,156 INFO sqlalchemy.engine.Engine [generated in 1.04284s] (('2021-11-16 20:20:00.000000', 7445.45, 7446.45, 7445.45, 7446.45, 80, 10), ('2021-11-16 20:25:00.000000', 7446.45, 7447.45, 7446.45, 7446.45, 80, 7), ('2021-11-16 20:30:00.000000', 7446.45, 7448.45, 7446.45, 7447.45, 80, 18), ('2021-11-16 20:35:00.000000', 7447.45, 7448.45, 7447.45, 7448.45, 80, 10), ('2021-11-16 20:40:00.000000', 7448.45, 7448.45, 7448.45, 7448.45, 80, 5), ('2021-11-16 20:45:00.000000', 7448.45, 7450.45, 7448.45, 7449.45, 80, 8), ('2021-11-16 20:50:00.000000', 7449.45, 7449.45, 7448.45, 7448.45, 80, 10), ('2021-11-16 20:55:00.000000', 7448.45, 7448.45, 7447.45, 7447.45, 80, 11)  ... displaying 10 of 99999 total bound parameter sets ...  ('2023-06-27 15:20:00.000000', 7104.71, 7104.71, 7102.71, 7104.71, 90, 27), ('2023-06-27 15:25:00.000